In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [2]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('CUDA version:', torch.version.cuda)
    print('Memory Usage:') 
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce GTX 960
CUDA version: 11.7
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [4]:
import numpy as np
import PIL
from PIL import Image
import datasets
import evaluate
import torch
import json
import codecs
import os
from os import sys

from transformers import SegformerImageProcessor, AutoModelForSemanticSegmentation, TrainingArguments, Trainer, EarlyStoppingCallback

/home/kate/miniconda3/envs/master/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
module_path = os.path.abspath(os.path.join('./src'))
if module_path not in sys.path:
    sys.path.append(module_path)
from data_prepossessing import create_datasets_for_plants, get_labels
from constants import *

In [6]:
checkpoint = "nvidia/mit-b0"
image_processor = SegformerImageProcessor.from_pretrained(checkpoint)
image_processor

/home/kate/miniconda3/envs/master/lib/python3.11/site-packages/transformers/models/segformer/image_processing_segformer.py:101: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


SegformerImageProcessor {
  "do_normalize": true,
  "do_reduce_labels": false,
  "do_rescale": true,
  "do_resize": true,
  "feature_extractor_type": "SegformerFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "SegformerImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 512,
    "width": 512
  }
}

In [7]:
def train_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels, return_tensors="pt")
    return inputs

In [8]:
metric = evaluate.load("mean_iou")

In [9]:
def compute_metrics(num_labels, eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = torch.nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if type(value) is np.ndarray:
                metrics[key] = value.tolist()
        return metrics

In [10]:
training_args = TrainingArguments(
    output_dir="segformer-b0-scene-parse-150",
    learning_rate=6e-5,
    num_train_epochs=50,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    eval_steps=100,
    logging_steps=1,
    remove_unused_columns=False,
    load_best_model_at_end=True,
    seed=seed,
)

In [11]:
def initialize_trainer(model, num_labels, train_ds, test_ds) :
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=test_ds,
        compute_metrics=lambda eval_pred: compute_metrics(num_labels, eval_pred),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )
    return trainer

In [12]:
def train_model_of_type_for_crop(model_type, crop):
    model_plant_names = [crop] + weed_plants
    train_ds, val_ds, test_ds = create_datasets_for_plants(model_plant_names, model_type, crop)

    print("Training subset number of images: " + str(train_ds.num_rows))
    print("Validation subset number of images: " + str(val_ds.num_rows))
    print("Test subset number of images: " + str(test_ds.num_rows))

    train_ds.set_transform(train_transforms)
    val_ds.set_transform(train_transforms)
    test_ds.set_transform(train_transforms)

    id2label, label2id = get_labels(crop, model_type)

    print('Number of classes:', len(id2label))
    print('id2label:', id2label)
    print('label2id:', label2id)

    model = AutoModelForSemanticSegmentation.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)
    trainer = initialize_trainer(model, len(id2label), train_ds, val_ds)
    trainer.train()

    # Save the trained model, so that it can be used for inference later.
    # Save the log history, so that it can be used for plotting later.
    trainer.save_model('models/' + model_type + '/' + crop)
    with open('models/' + model_type + '/' + crop + '/log_history.json', 'w') as file:
        log_history = trainer.state.log_history
        json.dump(log_history, file)

    test_metric = trainer.evaluate(test_ds)
    test_metric

    with open('models/' + model_type + '/' + crop + '/test_metric.json', 'w') as file:
        json.dump(test_metric, file)

In [13]:
# train_model_of_type_for_crop("multiclass", "broad_bean")
train_model_of_type_for_crop("multiclass", "sugar_beet")

['img_00071.png', 'img_00072.png', 'img_00074.png', 'img_00083.png', 'img_00084.png', 'img_00085.png', 'img_00151.png', 'img_00152.png', 'img_00153.png', 'img_00154.png', 'img_00155.png', 'img_00156.png', 'img_00210.png', 'img_00211.png', 'img_00212.png', 'img_00213.png', 'img_00214.png', 'img_00215.png', 'img_00216.png', 'img_00217.png', 'img_00218.png', 'img_00220.png', 'img_00221.png', 'img_00226.png', 'img_00227.png', 'img_00229.png', 'img_00248.png', 'img_00249.png', 'img_00250.png', 'img_00251.png', 'img_00254.png', 'img_00255.png', 'img_00256.png', 'img_00257.png', 'img_00258.png', 'img_00259.png', 'img_00260.png', 'img_00264.png', 'img_00265.png', 'img_00266.png', 'img_00267.png', 'img_00268.png', 'img_00269.png', 'img_00271.png', 'img_00293.png', 'img_00294.png', 'img_00295.png', 'img_00296.png', 'img_00297.png', 'img_00298.png', 'img_00299.png', 'img_00300.png', 'img_00301.png', 'img_00302.png', 'img_00303.png', 'img_00379.png', 'img_00380.png', 'img_00381.png', 'img_00382.pn

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.classifier.weight', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.batch_norm.bias', 'decode_head.linear_fuse.weight', 'decode_head.batch_norm.weight', 'decode_head.linear_c.0.proj.weight', 'decode_head.batch_norm.running_mean', 'decode_head.linear_c.2.proj.bias', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.linear_c.3.proj.bias', 'decode_head.classifier.bias', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 1/4750 [00:03<4:58:31,  3.77s/it]

{'loss': 2.4699, 'learning_rate': 5.998736842105264e-05, 'epoch': 0.01}


  0%|          | 2/4750 [00:06<4:04:28,  3.09s/it]

{'loss': 2.4603, 'learning_rate': 5.997473684210526e-05, 'epoch': 0.02}


...

 86%|████████▋ | 4100/4750 [5:51:13<28:09,  2.60s/it]

{'loss': 0.0078, 'learning_rate': 8.210526315789475e-06, 'epoch': 43.16}


                                                     
 86%|████████▋ | 4100/4750 [5:55:31<56:21,  5.20s/it]

{'eval_loss': 0.01571715623140335, 'eval_mean_iou': 0.4559616166004012, 'eval_mean_accuracy': 0.5065889948304223, 'eval_overall_accuracy': 0.9951927509713681, 'eval_per_category_iou': [0.00010729038141730594, 0.9959378956943508, 0.7999308733717438, 0.2610883339545285, 0.0, 0.4109110856230988, 0.0, 0.7223392260128951, 0.7652295448929995, 0.927117394423707, 0.650604177365095, 0.010002632271650434, 0.3842325618137294], 'eval_per_category_accuracy': [0.00010733068584308254, 0.9980547408383706, 0.9001979510331655, 0.27432935186998236, 0.0, 0.41816797767064195, 0.0, 0.8436602002646988, 0.8939711178893311, 0.9717197549345753, 0.7772825161516502, 0.010900745840504877, 0.49726524561672547], 'eval_runtime': 257.9984, 'eval_samples_per_second': 1.093, 'eval_steps_per_second': 0.182, 'epoch': 43.16}
{'train_runtime': 21331.2329, 'train_samples_per_second': 1.329, 'train_steps_per_second': 0.223, 'train_loss': 0.11343232588263275, 'epoch': 43.16}



100%|██████████| 48/48 [04:22<00:00,  5.48s/it]


In [14]:
# import subprocess
# from typing import NoReturn

# def shutdown_windows() -> NoReturn:
#     subprocess.run(["shutdown", "/s", "/t", "0"])

# shutdown_windows()